# Instalar librerias librerias

In [134]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip (21.8 MB)


# Lectura librerias

In [135]:
import pandas as pd 
import numpy as np
import json
import requests
import re
from datetime import datetime

from pandas_profiling import ProfileReport

pd.set_option('display.max_columns', None)

# Lectura datos

In [136]:
lst_days_zeros = ['0'+str(num) for num in range(1,10)]
lst_days_nonzeros = [str(num) for num in range(10,32)]
lst_days = lst_days_zeros+lst_days_nonzeros

In [137]:
df_lst_december = []
for i in lst_days:
  api_url = f"http://api.tvmaze.com/schedule/web?date=2020-12-{i}"
  response = requests.get(api_url)
  data_json = response.json()
  df_aux=pd.DataFrame(data_json)
  df_lst_december.append(df_aux)

In [138]:
df = pd.concat(df_lst_december)
df = df.reset_index()
df

,index,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,rating,image,summary,_links,_embedded
0,0,1979824,https://www.tvmaze.com/episodes/1979824/sim-fo...,Chanyeol's Episode 16,4,16.0,regular,2020-12-01,06:00,2020-11-30T21:00:00+00:00,16.0,{'average': None},None,<p><b>#ObtainedAConversationalSkill #WeSetUpAT...,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 41648, 'url': 'https://www.tvm..."
1,1,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12.0,{'average': None},None,None,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 52198, 'url': 'https://www.tvm..."
2,2,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29.0,{'average': None},{'medium': 'https://static.tvmaze.com/uploads/...,None,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 52933, 'url': 'https://www.tvm..."
3,3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24.0,{'average': None},None,None,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 51336, 'url': 'https://www.tvm..."
4,4,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80.0,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,8.0,{'average': None},None,None,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 54033, 'url': 'https://www.tvm..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3030,125,2005098,https://www.tvmaze.com/episodes/2005098/carl-w...,The Nuclear Option,2,8.0,regular,2020-12-31,21:00,2021-01-01T02:00:00+00:00,60.0,{'average': None},{'medium': 'https://static.tvmaze.com/uploads/...,<p>The war heats up as the Duncans deploy the ...,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 39441, 'url': 'https://www.tvm..."
3031,126,2005099,https://www.tvmaze.com/episodes/2005099/carl-w...,Mothers and Their Sons,2,9.0,regular,2020-12-31,21:00,2021-01-01T02:00:00+00:00,60.0,{'average': None},{'medium': 'https://static.tvmaze.com/uploads/...,<p>Junior and Sonya make landfall in Cuba. Chi...,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 39441, 'url': 'https://www.tvm..."
3032,127,2005100,https://www.tvmaze.com/episodes/2005100/carl-w...,I Don't Want to Lose My Queen,2,10.0,regular,2020-12-31,21:00,2021-01-01T02:00:00+00:00,60.0,{'average': None},{'medium': 'https://static.tvmaze.com/uploads/...,<p>Brother X begins to unravel. Sasha and Elij...,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 39441, 'url': 'https://www.tvm..."
3033,128,2005101,https://www.tvmaze.com/episodes/2005101/carl-w...,Deploy the Drones,2,11.0,regular,2020-12-31,21:00,2021-01-01T02:00:00+00:00,60.0,{'average': None},{'medium': 'https://static.tvmaze.com/uploads/...,<p>The war between the Duncans and Brother X i...,{'self': {'href': 'https://api.tvmaze.com/epis...,"{'show': {'id': 39441, 'url': 'https://www.tvm..."


In [139]:
# Exportar como json
df.to_json(r'/content/drive/MyDrive/Lulobank/json/BroadCast_shows.json')

# Tratamiento de bases de datos

## Main table - episodes

In [140]:
#Función para extracción de rating promedio.
def extract_rating(x):
  try:
    return list(x.values())[0]
  except:
    return None
#Función para limpieza de textos.
def clean_text(text):
  try:
    return re.sub("</?p>|</?b>|</?i>","",text)
  except:
    return text

In [141]:
# Transformar a formato fecha %Y-%m-%d para facilitar la lectura de en SQL
df['airdate']=pd.to_datetime(df['airdate'],format='%Y-%m-%d')
df['airstamp']=pd.to_datetime(df['airstamp']).dt.strftime('%Y-%m-%d')
# Extraer el rating del json de rating promedio
df['rating'] = df['rating'].apply(lambda x: extract_rating(x))
# Extraer el rating del json de rating promedio
df['summary'] = df['summary'].apply(lambda x: clean_text(x))
# Crear llave para unir con las demás tablas.
df = df.reset_index(drop=True)
df= df.reset_index()
df = df.rename(columns={'level_0':'episode_ids'})


In [142]:
# Limpieza de tabla de episodios principal
df_episodes_clean = df[['episode_ids','id', 'url', 'name', 'season', 'number', 'type', 'airdate', 'airtime','airstamp', 'runtime', 'rating', 'summary']]
df_episodes_clean

,episode_ids,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,rating,summary
0,0,1979824,https://www.tvmaze.com/episodes/1979824/sim-fo...,Chanyeol's Episode 16,4,16.0,regular,2020-12-01,06:00,2020-11-30,16.0,NaN,#ObtainedAConversationalSkill #WeSetUpATent♥ #...
1,1,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2.0,regular,2020-12-01,,2020-12-01,12.0,NaN,None
2,2,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6.0,regular,2020-12-01,,2020-12-01,29.0,NaN,None
3,3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9.0,regular,2020-12-01,10:00,2020-12-01,24.0,NaN,None
4,4,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80.0,regular,2020-12-01,10:00,2020-12-01,8.0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3030,3030,2005098,https://www.tvmaze.com/episodes/2005098/carl-w...,The Nuclear Option,2,8.0,regular,2020-12-31,21:00,2021-01-01,60.0,NaN,The war heats up as the Duncans deploy the nuc...
3031,3031,2005099,https://www.tvmaze.com/episodes/2005099/carl-w...,Mothers and Their Sons,2,9.0,regular,2020-12-31,21:00,2021-01-01,60.0,NaN,Junior and Sonya make landfall in Cuba. Chippy...
3032,3032,2005100,https://www.tvmaze.com/episodes/2005100/carl-w...,I Don't Want to Lose My Queen,2,10.0,regular,2020-12-31,21:00,2021-01-01,60.0,NaN,Brother X begins to unravel. Sasha and Elijah ...
3033,3033,2005101,https://www.tvmaze.com/episodes/2005101/carl-w...,Deploy the Drones,2,11.0,regular,2020-12-31,21:00,2021-01-01,60.0,NaN,The war between the Duncans and Brother X is f...


### Limpieza datos inconsistentes en season

In [143]:
# Revisar frecuencia de datos raros
df_episodes_clean['season'].value_counts(normalize=True)

1       0.613839
2       0.115321
2020    0.072488
3       0.054366
4       0.051400
5       0.016145
7       0.013509
6       0.011532
18      0.007578
9       0.007578
10      0.006260
31      0.006260
8       0.005931
12      0.002306
14      0.001977
19      0.001977
51      0.001647
15      0.001647
27      0.001318
48      0.001318
30      0.001318
11      0.000988
17      0.000988
13      0.000659
54      0.000659
16      0.000329
2019    0.000329
23      0.000329
Name: season, dtype: float64

In [144]:
df_episodes_clean['season'] = df_episodes_clean['season'].replace({2020:None,2019:None})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [145]:
df_episodes_clean['season'].value_counts(normalize=True)

1.0     0.662047
2.0     0.124378
3.0     0.058635
4.0     0.055437
5.0     0.017413
7.0     0.014570
6.0     0.012438
18.0    0.008173
9.0     0.008173
10.0    0.006752
31.0    0.006752
8.0     0.006397
12.0    0.002488
19.0    0.002132
14.0    0.002132
51.0    0.001777
15.0    0.001777
27.0    0.001421
48.0    0.001421
30.0    0.001421
17.0    0.001066
11.0    0.001066
13.0    0.000711
54.0    0.000711
16.0    0.000355
23.0    0.000355
Name: season, dtype: float64

## Imagenes de episodio

In [146]:
# Abrir la columna de imagenes para volverlo una tabla.
lst_embeddeding_images_episode = df.image.to_list()
# Volver la lista de jsons en un dataframe.
df_images_episode = pd.json_normalize(lst_embeddeding_images_episode).reset_index()
# Crear un id para unirlo a la tabla de episodios
df_images_episode_clean=df_images_episode.rename(columns={'index':'episode_ids'})
df_images_episode_clean = df_images_episode_clean.dropna()

In [147]:
df_images_episode_clean

,episode_ids,medium,original
2,2,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
12,12,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
13,13,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
14,14,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
15,15,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
...,...,...,...
3030,3030,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
3031,3031,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
3032,3032,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
3033,3033,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...


## Links episodios

In [148]:
# Abrir la columna de _links para volverlo una tabla 
lst_link = df['_links'].to_list()
# Volver la lista de jsons en un dataframe
df_link = pd.json_normalize(lst_link).reset_index()
# Crear un id para unirlo a la tabla de episodios
df_link = df_link.rename(columns={'index':'episode_ids'})

In [149]:
def extractid(x):
  '''
  x : url para extraer el numero de episodio
  return el numero de episodio que contiene la tabla de links, en el caso que no haya link no devuelve nada.
  Función para extraer el id del episodio de los urls de la tabla de links.
  '''
  try:
    return x.split('/')[-1]
  except:
    return None



In [150]:
# Renombrar las columnas de las tablas para que tenga un id
df_link = df_link.rename(columns = {'self.href':'episode_id'})
# Aplicar la función que obtiene el id del episodio
df_link['episode_id'] = df_link['episode_id'].apply(lambda x: extractid(x))
df_link

,episode_ids,episode_id
0,0,1979824
1,1,1979222
2,2,2008027
3,3,1964565
4,4,2052503
...,...,...
3030,3030,2005098
3031,3031,2005099
3032,3032,2005100
3033,3033,2005101


## Shows table

In [151]:
# Abrir la columna de embedding para volverlo una tabla 
lst_embeddeding = df['_embedded'].to_list()
# Volver la lista de jsons en un dataframe
lst_embedded = pd.DataFrame.from_records(lst_embeddeding)
# Volver un dataframe quitando la llave que contiene el show
df_embedding = pd.DataFrame(lst_embedded['show'].to_list())


In [152]:
# Exportar como json
df_embedding.to_json(r'/content/drive/MyDrive/Lulobank/json/ShowsGeneralInformation.json')

In [153]:
# Volver el rating un numero en vez de un json
df_embedding['rating'] = df_embedding['rating'].apply(lambda x: extract_rating(x))
# Limpiar caracteres especiales de los resumenes
df_embedding['summary'] = df_embedding['summary'].apply(lambda x: clean_text(x))
# Volver la fecha de actualización de UNIXTIME en formato fecha %Y-%m-%d
df_embedding['updated'] = df_embedding['updated'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
# Quitar duplicados
df_embedding = df_embedding.drop_duplicates(subset=['id', 'url', 'name'])
# Crear la llave para unirlo con los episodios.
df_embedding = df_embedding.reset_index()
df_embedding = df_embedding.rename(columns={'index':'episode_ids'})
df_embedding

,episode_ids,id,url,name,type,language,genres,status,runtime,averageRuntime,premiered,ended,officialSite,schedule,rating,weight,network,webChannel,dvdCountry,externals,image,summary,updated,_links
0,0,41648,https://www.tvmaze.com/shows/41648/sim-for-you,Sim for You,Reality,Korean,[],Running,16.0,16.0,2019-03-25,None,https://www.vlive.tv/video/121637,"{'time': '', 'days': ['Monday', 'Wednesday', '...",NaN,69,None,"{'id': 122, 'name': 'V LIVE', 'country': {'nam...",None,"{'tvrage': None, 'thetvdb': 361541, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,Sim for You is a reality series that chronicle...,2020-12-20,{'self': {'href': 'https://api.tvmaze.com/show...
1,1,52198,https://www.tvmaze.com/shows/52198/kotiki,Котики,Scripted,Russian,[Comedy],Ended,12.0,12.0,2020-11-30,2020-12-11,http://epic-media.ru/project/kotiki,"{'time': '10:00', 'days': ['Monday', 'Tuesday'...",NaN,15,None,"{'id': 510, 'name': 'Epic Media', 'country': {...",None,"{'tvrage': None, 'thetvdb': 392682, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,None,2021-11-22,{'self': {'href': 'https://api.tvmaze.com/show...
2,2,52933,https://www.tvmaze.com/shows/52933/lab-s-anton...,LAB с Антоном Беляевым,Documentary,Russian,[Music],To Be Determined,26.0,25.0,2019-12-17,None,https://premier.one/show/lab-laboratoriya-muzy...,"{'time': '23:45', 'days': ['Saturday']}",NaN,23,"{'id': 308, 'name': 'ТНТ', 'country': {'name':...","{'id': 381, 'name': 'КиноПоиск HD', 'country':...",None,"{'tvrage': None, 'thetvdb': None, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,Russian music artists reveal themselves from u...,2022-05-31,{'self': {'href': 'https://api.tvmaze.com/show...
3,3,51336,https://www.tvmaze.com/shows/51336/core-sense,Core Sense,Animation,Chinese,"[Action, Anime, Science-Fiction]",Running,24.0,24.0,2020-10-13,None,https://www.bilibili.com/bangumi/media/md28223064,"{'time': '10:00', 'days': ['Tuesday']}",NaN,38,None,"{'id': 51, 'name': 'Bilibili', 'country': {'na...",None,"{'tvrage': None, 'thetvdb': None, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,"The power of beginnings, the energy of the cor...",2020-11-05,{'self': {'href': 'https://api.tvmaze.com/show...
4,4,54033,https://www.tvmaze.com/shows/54033/wu-shen-zhu...,Wu Shen Zhu Zai,Animation,Chinese,"[Action, Adventure, Anime, Fantasy]",Running,8.0,8.0,2020-03-08,None,https://v.qq.com/detail/m/7q544xyrava3vxf.html,"{'time': '10:00', 'days': ['Tuesday', 'Sunday']}",NaN,81,None,"{'id': 104, 'name': 'Tencent QQ', 'country': {...",None,"{'tvrage': None, 'thetvdb': 379070, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,"The protagonist Qin Chen, who was originally t...",2022-04-08,{'self': {'href': 'https://api.tvmaze.com/show...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,2989,51826,https://www.tvmaze.com/shows/51826/strange-heart,Strange Heart,Scripted,Tagalog,"[Drama, Romance]",Ended,25.0,26.0,2020-10-26,2020-12-31,https://www.youtube.com/playlist?list=PLJPWxm3...,"{'time': '', 'days': []}",NaN,13,None,"{'id': 21, 'name': 'YouTube', 'country': None,...",None,"{'tvrage': None, 'thetvdb': None, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,"They said, there's nothing wrong about love. B...",2021-03-21,{'self': {'href': 'https://api.tvmaze.com/show...
619,2999,52847,https://www.tvmaze.com/shows/52847/three-men-f...,Three Men Four Wheels,Reality,English,[],Running,NaN,NaN,2020-12-31,None,https://www.discoveryplus.co.uk/show/three-men...,"{'time': '', 'days': []}",NaN,35,None,"{'id': 173, 'name': 'discovery+', 'country': N...",None,"{'tvrage': None, 'thetvdb': 394277, 'imdb': 't...",{'medium': 'https://static.tvmaze.com/uploads/...,"Antique and classic car dealer Drew Pritchard,...",2021-01-20,{'self': {'href': 'https://api.tvmaze.com/show...
620,3011,53581,https://www.tvmaze.com/shows/53581/thursday-night,Thursday Night,Talk Show,Arabic,"[Comedy, Fam

In [154]:
df_embedding

,episode_ids,id,url,name,type,language,genres,status,runtime,averageRuntime,premiered,ended,officialSite,schedule,rating,weight,network,webChannel,dvdCountry,externals,image,summary,updated,_links
0,0,41648,https://www.tvmaze.com/shows/41648/sim-for-you,Sim for You,Reality,Korean,[],Running,16.0,16.0,2019-03-25,None,https://www.vlive.tv/video/121637,"{'time': '', 'days': ['Monday', 'Wednesday', '...",NaN,69,None,"{'id': 122, 'name': 'V LIVE', 'country': {'nam...",None,"{'tvrage': None, 'thetvdb': 361541, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,Sim for You is a reality series that chronicle...,2020-12-20,{'self': {'href': 'https://api.tvmaze.com/show...
1,1,52198,https://www.tvmaze.com/shows/52198/kotiki,Котики,Scripted,Russian,[Comedy],Ended,12.0,12.0,2020-11-30,2020-12-11,http://epic-media.ru/project/kotiki,"{'time': '10:00', 'days': ['Monday', 'Tuesday'...",NaN,15,None,"{'id': 510, 'name': 'Epic Media', 'country': {...",None,"{'tvrage': None, 'thetvdb': 392682, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,None,2021-11-22,{'self': {'href': 'https://api.tvmaze.com/show...
2,2,52933,https://www.tvmaze.com/shows/52933/lab-s-anton...,LAB с Антоном Беляевым,Documentary,Russian,[Music],To Be Determined,26.0,25.0,2019-12-17,None,https://premier.one/show/lab-laboratoriya-muzy...,"{'time': '23:45', 'days': ['Saturday']}",NaN,23,"{'id': 308, 'name': 'ТНТ', 'country': {'name':...","{'id': 381, 'name': 'КиноПоиск HD', 'country':...",None,"{'tvrage': None, 'thetvdb': None, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,Russian music artists reveal themselves from u...,2022-05-31,{'self': {'href': 'https://api.tvmaze.com/show...
3,3,51336,https://www.tvmaze.com/shows/51336/core-sense,Core Sense,Animation,Chinese,"[Action, Anime, Science-Fiction]",Running,24.0,24.0,2020-10-13,None,https://www.bilibili.com/bangumi/media/md28223064,"{'time': '10:00', 'days': ['Tuesday']}",NaN,38,None,"{'id': 51, 'name': 'Bilibili', 'country': {'na...",None,"{'tvrage': None, 'thetvdb': None, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,"The power of beginnings, the energy of the cor...",2020-11-05,{'self': {'href': 'https://api.tvmaze.com/show...
4,4,54033,https://www.tvmaze.com/shows/54033/wu-shen-zhu...,Wu Shen Zhu Zai,Animation,Chinese,"[Action, Adventure, Anime, Fantasy]",Running,8.0,8.0,2020-03-08,None,https://v.qq.com/detail/m/7q544xyrava3vxf.html,"{'time': '10:00', 'days': ['Tuesday', 'Sunday']}",NaN,81,None,"{'id': 104, 'name': 'Tencent QQ', 'country': {...",None,"{'tvrage': None, 'thetvdb': 379070, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,"The protagonist Qin Chen, who was originally t...",2022-04-08,{'self': {'href': 'https://api.tvmaze.com/show...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,2989,51826,https://www.tvmaze.com/shows/51826/strange-heart,Strange Heart,Scripted,Tagalog,"[Drama, Romance]",Ended,25.0,26.0,2020-10-26,2020-12-31,https://www.youtube.com/playlist?list=PLJPWxm3...,"{'time': '', 'days': []}",NaN,13,None,"{'id': 21, 'name': 'YouTube', 'country': None,...",None,"{'tvrage': None, 'thetvdb': None, 'imdb': None}",{'medium': 'https://static.tvmaze.com/uploads/...,"They said, there's nothing wrong about love. B...",2021-03-21,{'self': {'href': 'https://api.tvmaze.com/show...
619,2999,52847,https://www.tvmaze.com/shows/52847/three-men-f...,Three Men Four Wheels,Reality,English,[],Running,NaN,NaN,2020-12-31,None,https://www.discoveryplus.co.uk/show/three-men...,"{'time': '', 'days': []}",NaN,35,None,"{'id': 173, 'name': 'discovery+', 'country': N...",None,"{'tvrage': None, 'thetvdb': 394277, 'imdb': 't...",{'medium': 'https://static.tvmaze.com/uploads/...,"Antique and classic car dealer Drew Pritchard,...",2021-01-20,{'self': {'href': 'https://api.tvmaze.com/show...
620,3011,53581,https://www.tvmaze.com/shows/53581/thursday-night,Thursday Night,Talk Show,Arabic,"[Comedy, Fam

In [155]:
# Limpieza de los campos que se van a volver tablas debido a que son json y pueden contener información importante
df_embedding_clean = df_embedding[['episode_ids','id', 'url', 'name', 'type', 'language','status', 'runtime','averageRuntime', 'premiered', 'ended', 'officialSite','rating', 'weight','summary', 'updated']].sort_values(by='id')

In [156]:
df_embedding_clean = df_embedding_clean.reset_index()
df_embedding_clean = df_embedding_clean.rename(columns={'index':'shows_id'})
df_embedding_clean

,shows_id,episode_ids,id,url,name,type,language,status,runtime,averageRuntime,premiered,ended,officialSite,rating,weight,summary,updated
0,350,680,802,https://www.tvmaze.com/shows/802/wwe-monday-ni...,WWE Monday Night RAW,Sports,English,Running,180.0,181.0,1993-01-11,None,http://www.wwe.com/,7.5,93,WWE Monday Night RAW is World Wrestling Entert...,2022-06-18
1,279,539,1596,https://www.tvmaze.com/shows/1596/ufc-fight-night,UFC Fight Night,Sports,English,Running,120.0,193.0,2005-08-06,None,http://www.ufc.com/,7.7,97,UFC Fight Night is a part of the Ultimate Figh...,2022-06-26
2,463,1535,1825,https://www.tvmaze.com/shows/1825/the-expanse,The Expanse,Scripted,English,Ended,NaN,55.0,2015-12-14,2022-01-14,https://www.amazon.com/dp/B07YL9WK1S/,8.8,99,A thriller set two hundred years in the future...,2022-04-17
3,110,175,2266,https://www.tvmaze.com/shows/2266/wwe-nxt,WWE NXT,Sports,English,Running,120.0,76.0,2010-02-23,None,http://www.wwe.com/inside/networkschedule,7.2,91,"Each Wednesday at 8:00 p.m. ET, WWE Superstars...",2022-06-22
4,65,94,2504,https://www.tvmaze.com/shows/2504/goede-tijden...,"Goede Tijden, Slechte Tijden",Scripted,Dutch,Running,23.0,25.0,1990-10-01,None,http://gtst.nl/#!/,NaN,80,None,2022-06-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,472,1609,62127,https://www.tvmaze.com/shows/62127/30-days-wit...,30 Days With: Ryan Garcia,Documentary,English,Ended,18.0,19.0,2020-12-16,2021-01-13,https://www.youtube.com/playlist?list=PLf9hgds...,NaN,0,30 Days With: Ryan Garcia follows the undefeat...,2022-05-21
619,412,1147,62306,https://www.tvmaze.com/shows/62306/rainbow-high,Rainbow High,Animation,English,Running,6.0,6.0,2020-10-02,None,https://www.youtube.com/c/RainbowHigh,NaN,30,"Legend says at the end of the rainbow, there's...",2022-06-12
620,244,478,62418,https://www.tvmaze.com/shows/62418/trailer-par...,Trailer Park Boys: Park After Dark,Talk Show,English,Running,30.0,30.0,2019-04-05,None,https://www.swearnet.com/shows/park-after-dark,NaN,42,"Hang out with Ricky, Julian and Bubbles in Ric...",2022-06-24
621,425,1213,62545,https://www.tvmaze.com/shows/62545/ww2-war-aga...,WW2: War Against Humanity,Documentary,English,Running,NaN,17.0,2019-03-06,None,None,NaN,42,"War Against Humanity is a sub-series of ""World...",2022-06-22


## Generos de los shows

In [157]:
# Abrir la columna de embedding para volverlo una tabla.
lst_embeddeding_genres = df_embedding['genres'].to_list()
# Volver dummies los generos.
df_shows_genres = pd.get_dummies(pd.DataFrame(lst_embeddeding_genres))
df_shows_genres.columns = df_shows_genres.columns.str.split("_").str[-1]
# Agrupar los generos que esten repetidos en columnas.
df_shows_genres = df_shows_genres.groupby(by=df_shows_genres.columns, axis=1).sum()
# Agregar cantidad de generos por show.
df_shows_genres['quantity']=df_shows_genres.sum(axis=1)
# Traer la combinación de los generos.
df_shows_genres['combination'] = df_embedding['genres']
# Crear la llave para unirlo con los episodios.
df_shows_genres = df_shows_genres.reset_index()
df_shows_genres = df_shows_genres.rename(columns={'index':'shows_id'})


In [158]:
df_shows_genres

,shows_id,Action,Adult,Adventure,Anime,Children,Comedy,Crime,DIY,Drama,Family,Fantasy,Food,History,Horror,Legal,Medical,Music,Mystery,Nature,Romance,Science-Fiction,Sports,Supernatural,Thriller,Travel,War,quantity,combination
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[]
1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,[Comedy]
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,[Music]
3,3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,"[Action, Anime, Science-Fiction]"
4,4,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,"[Action, Adventure, Anime, Fantasy]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,618,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,"[Drama, Romance]"
619,619,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[]
620,620,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,"[Comedy, Family, Music]"
621,621,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[]


In [159]:
df_shows_genres.drop(columns=['quantity','combination']).sum()

shows_id           193753
Action                 38
Adult                   1
Adventure              35
Anime                  30
Children               40
Comedy                179
Crime                  40
DIY                     2
Drama                 169
Family                 24
Fantasy                35
Food                   19
History                19
Horror                 15
Legal                   2
Medical                 4
Music                  26
Mystery                31
Nature                  6
Romance                89
Science-Fiction        22
Sports                 21
Supernatural           15
Thriller               28
Travel                  8
War                     6
dtype: int64

## Horarios de los shows

In [160]:
# Abrir la columna de embedding para volverlo una tabla.
lst_embeddeding_schedule = df_embedding['schedule'].to_list()
# Volver la lista de jsons en un dataframe.
df_schedule = pd.DataFrame.from_records(lst_embeddeding_schedule)
# Dumificar los dias que se emiten los shows.
schedule_days = pd.get_dummies(df_schedule.days.apply(pd.Series).stack()).sum(level=0)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [161]:
# Crear el id para cruzarlo con los shows.
schedule_days = schedule_days.reset_index()
df_schedule = df_schedule.reset_index()

In [162]:
# Unir los shows con las horas de los shows.
schedules_days_complete = pd.merge(df_schedule,schedule_days,how='left',on='index').fillna(0)
# Quitar las columnas que son irrelevantes y que ya se hicieron la transformación
schedules_days_complete = schedules_days_complete.drop(columns=['index','days'])
# Cambiar los espacios vacios por 0.
schedules_days_complete.time = schedules_days_complete.time.replace('',0)
# Se cambio a 0 los espacios vacios para poder filtrar los shows que no contienen información de los dias de emisión o de las horas.
df_schedules_clean = schedules_days_complete[schedules_days_complete.sum(axis=1)>0].reset_index()
df_schedules_clean.time = df_schedules_clean.time.replace(0,'')
# Re organizar la tabla
df_schedules_clean = df_schedules_clean.rename(columns={'index':'shows_id'})
df_schedules_clean = df_schedules_clean[['shows_id', 'time','Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday',  'Saturday', 'Sunday']]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [163]:
df_schedules_clean

,shows_id,time,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0,,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1,10:00,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,2,23:45,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,10:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,10:00,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
439,612,00:00,0.0,0.0,0.0,1.0,0.0,0.0,0.0
440,613,,1.0,0.0,0.0,0.0,0.0,0.0,0.0
441,614,,0.0,0.0,0.0,0.0,1.0,0.0,0.0
442,620,23:00,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Imagenes de los shows

In [164]:
# Abrir la columna de imagenes para volverlo una tabla.
lst_embeddeding_images = df_embedding.image.to_list()
# Volver la lista de jsons en un dataframe.
df_images = pd.json_normalize(lst_embeddeding_images).reset_index()
# Crear un id para unirlo a la tabla de episodios
df_images=df_images.rename(columns={'index':'shows_id'})

In [165]:
df_images

,shows_id,medium,original
0,0,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
1,1,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
2,2,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
3,3,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
4,4,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
...,...,...,...
618,618,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
619,619,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
620,620,https://static.tvmaze.com/uploads/images/mediu...,https://static.tvmaze.com/uploads/images/origi...
621,621,NaN,NaN


## Canales

In [199]:
# Abrir la columna de canales para volverlo una tabla.
lst_embeddeding_webChannel = df_embedding['webChannel'].to_list()
# Volver la lista de jsons en un dataframe.
df_webChannel = pd.json_normalize(lst_embeddeding_webChannel).reset_index()
# Una para la información de los canales y otra para la conexión entre los canales y los shows
df_webChannel = df_webChannel.rename(columns={'index':'shows_id','id':'channel_id'})
df_webChannel_connection = df_webChannel[['shows_id','channel_id']]
# Quitar los canales duplicados de la tabla de canales
df_webChannel_clean = df_webChannel.drop(columns=['shows_id']).drop_duplicates()
df_webChannel_clean = pd.merge(df_webChannel_clean,df_webChannel_connection, how  = 'left', on = 'channel_id')

In [200]:
df_webChannel_clean


,channel_id,name,officialSite,country.name,country.code,country.timezone,country,shows_id
0,122.0,V LIVE,https://www.vlive.tv/home,"Korea, Republic of",KR,Asia/Seoul,NaN,0
1,122.0,V LIVE,https://www.vlive.tv/home,"Korea, Republic of",KR,Asia/Seoul,NaN,316
2,510.0,Epic Media,None,Russian Federation,RU,Asia/Kamchatka,NaN,1
3,381.0,КиноПоиск HD,None,Russian Federation,RU,Asia/Kamchatka,NaN,2
4,381.0,КиноПоиск HD,None,Russian Federation,RU,Asia/Kamchatka,NaN,474
...,...,...,...,...,...,...,...,...
618,436.0,discovery+,None,United States,US,America/New_York,NaN,591
619,471.0,DR TV,https://dr.dk/drtv,Denmark,DK,Europe/Copenhagen,NaN,594
620,366.0,Okko,None,Russian Federation,RU,Asia/Kamchatka,NaN,598
621,447.0,GAIN,None,Turkey,TR,Europe/Istanbul,NaN,602


## Links externos Shows

In [168]:
# Abrir la columna de links externos para volverlo una tabla.
lst_embeddeding_external = df_embedding['externals'].to_list()
# Volver la lista de jsons en un dataframe.
df_external = pd.json_normalize(lst_embeddeding_external).reset_index()
df_external = df_external.rename(columns={'index':'shows_id'})


In [169]:
df_external

,shows_id,tvrage,thetvdb,imdb
0,0,NaN,361541.0,None
1,1,NaN,392682.0,None
2,2,NaN,NaN,None
3,3,NaN,NaN,None
4,4,NaN,379070.0,None
...,...,...,...,...
618,618,NaN,NaN,None
619,619,NaN,394277.0,tt13750872
620,620,NaN,396705.0,None
621,621,NaN,NaN,None


## Links episodios previos y siguientes

In [170]:
# Abrir la columna de links externos para volverlo una tabla.
lst_embeddeding_links = df_embedding['_links'].to_list()
# Volver la lista de jsons en un dataframe.
df_links = pd.json_normalize(lst_embeddeding_links).reset_index()
# Crear un id para unirlo a la tabla de episodios
df_links = df_links.rename(columns={'index':'shows_id'})
# Renombrar las columnas para mejor entendimiento y facilidad de usarlo en queries
df_links = df_links.rename(columns = {'self.href':'episode_id','previousepisode.href':'previousepisode_id','nextepisode.href':'nextepisode_id'})




In [171]:
# Extraer el episodio del show, del episodio previo y del siguiente
df_links['episode_id'] = df_links['episode_id'].apply(lambda x: extractid(x))
df_links['previousepisode_id'] = df_links['previousepisode_id'].apply(lambda x: extractid(x))
df_links['nextepisode_id'] = df_links['nextepisode_id'].apply(lambda x: extractid(x))

df_links

,shows_id,episode_id,previousepisode_id,nextepisode_id
0,0,41648,1988862,None
1,1,52198,1986873,None
2,2,52933,2245512,None
3,3,51336,1964569,None
4,4,54033,2309421,2309422
...,...,...,...,...
618,618,51826,2050241,None
619,619,52847,2001674,None
620,620,53581,2046124,None
621,621,59380,2234297,None


# Profiling

In [172]:
ProfileReport(df_episodes_clean,title='Shows Broadcasting Information').to_file(output_file = '/content/drive/MyDrive/Lulobank/profiling/Shows_Broadcasting_Information.html')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [173]:
df_episodes_clean.to_csv('/content/drive/MyDrive/Lulobank/model/Shows_Broadcasting_Information.csv')

In [174]:
ProfileReport(df_embedding_clean,title='Shows General Information').to_file(output_file = '/content/drive/MyDrive/Lulobank/profiling/Shows_General_Information.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [175]:
df_embedding_clean.to_csv('/content/drive/MyDrive/Lulobank/model/Shows_General_Information.csv')

In [176]:
ProfileReport(df_schedules_clean,title='Horario de los shows').to_file(output_file = '/content/drive/MyDrive/Lulobank/profiling/Schedules_Shows.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [177]:
df_schedules_clean.to_csv('/content/drive/MyDrive/Lulobank/model/Schedules_Shows.csv')

In [178]:
ProfileReport(df_webChannel_clean,title='WebChannel Information').to_file(output_file = '/content/drive/MyDrive/Lulobank/profiling/WebChannel_Shows.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [202]:
df_webChannel_clean.to_csv('/content/drive/MyDrive/Lulobank/model/WebChannel_Shows.csv')

In [180]:
ProfileReport(df_external,title='External links Information').to_file(output_file = '/content/drive/MyDrive/Lulobank/profiling/External_links_Shows.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [181]:
df_external.to_csv('/content/drive/MyDrive/Lulobank/model/External_links_Shows.csv')

In [182]:
ProfileReport(df_links,title='Links episodios').to_file(output_file = '/content/drive/MyDrive/Lulobank/profiling/link_episodios.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [183]:
df_links.to_csv('/content/drive/MyDrive/Lulobank/model/link_episodios.csv')

In [184]:
ProfileReport(df_shows_genres,title='Generos de las series').to_file(output_file = '/content/drive/MyDrive/Lulobank/profiling/genres_shows.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [194]:
df_shows_genres.to_csv('/content/drive/MyDrive/Lulobank/model/genres_shows.csv')

# Exportar to sqlite

In [186]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///LuloBankTest.db', echo=True)
sqlite_connection = engine.connect()

In [187]:
df_episodes_clean.to_sql("BroadCastShows", sqlite_connection, if_exists='fail')

2022-06-27 20:43:30,703 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("BroadCastShows")
2022-06-27 20:43:30,705 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:30,713 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("BroadCastShows")
2022-06-27 20:43:30,714 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:30,728 INFO sqlalchemy.engine.Engine 
CREATE TABLE "BroadCastShows" (
	"index" BIGINT, 
	episode_ids BIGINT, 
	id BIGINT, 
	url TEXT, 
	name TEXT, 
	season FLOAT, 
	number FLOAT, 
	type TEXT, 
	airdate DATETIME, 
	airtime TEXT, 
	airstamp TEXT, 
	runtime FLOAT, 
	rating FLOAT, 
	summary TEXT
)


2022-06-27 20:43:30,732 INFO sqlalchemy.engine.Engine [no key 0.00414s] ()
2022-06-27 20:43:30,744 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 20:43:30,746 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_BroadCastShows_index" ON "BroadCastShows" ("index")
2022-06-27 20:43:30,747 INFO sqlalchemy.engine.Engine [no key 0.00181s] ()
2022-06-27 20:43:30,759 INFO

In [188]:
df_embedding_clean.to_sql("shows", sqlite_connection)

2022-06-27 20:43:30,949 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("shows")
2022-06-27 20:43:30,952 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:30,958 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("shows")
2022-06-27 20:43:30,960 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:30,972 INFO sqlalchemy.engine.Engine 
CREATE TABLE shows (
	"index" BIGINT, 
	shows_id BIGINT, 
	episode_ids BIGINT, 
	id BIGINT, 
	url TEXT, 
	name TEXT, 
	type TEXT, 
	language TEXT, 
	status TEXT, 
	runtime FLOAT, 
	"averageRuntime" FLOAT, 
	premiered TEXT, 
	ended TEXT, 
	"officialSite" TEXT, 
	rating FLOAT, 
	weight BIGINT, 
	summary TEXT, 
	updated TEXT
)


2022-06-27 20:43:30,975 INFO sqlalchemy.engine.Engine [no key 0.00263s] ()
2022-06-27 20:43:30,994 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 20:43:30,996 INFO sqlalchemy.engine.Engine CREATE INDEX ix_shows_index ON shows ("index")
2022-06-27 20:43:30,999 INFO sqlalchemy.engine.Engine [no key 0.00233s] ()

In [189]:
df_schedules_clean.to_sql("Schedules", sqlite_connection)

2022-06-27 20:43:31,091 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Schedules")
2022-06-27 20:43:31,095 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:31,100 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Schedules")
2022-06-27 20:43:31,102 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:31,109 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Schedules" (
	"index" BIGINT, 
	shows_id BIGINT, 
	time TEXT, 
	"Monday" FLOAT, 
	"Tuesday" FLOAT, 
	"Wednesday" FLOAT, 
	"Thursday" FLOAT, 
	"Friday" FLOAT, 
	"Saturday" FLOAT, 
	"Sunday" FLOAT
)


2022-06-27 20:43:31,120 INFO sqlalchemy.engine.Engine [no key 0.01079s] ()
2022-06-27 20:43:31,136 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 20:43:31,138 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Schedules_index" ON "Schedules" ("index")
2022-06-27 20:43:31,140 INFO sqlalchemy.engine.Engine [no key 0.00192s] ()
2022-06-27 20:43:31,152 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 20:43:31,158 INFO sqla

In [190]:
df_webChannel_clean.to_sql("WebChannel", sqlite_connection)

2022-06-27 20:43:31,221 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("WebChannel")
2022-06-27 20:43:31,226 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:31,232 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("WebChannel")
2022-06-27 20:43:31,234 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:31,240 INFO sqlalchemy.engine.Engine 
CREATE TABLE "WebChannel" (
	"index" BIGINT, 
	channel_id FLOAT, 
	name TEXT, 
	"officialSite" TEXT, 
	"country.name" TEXT, 
	"country.code" TEXT, 
	"country.timezone" TEXT, 
	country FLOAT
)


2022-06-27 20:43:31,242 INFO sqlalchemy.engine.Engine [no key 0.00232s] ()
2022-06-27 20:43:31,255 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 20:43:31,256 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_WebChannel_index" ON "WebChannel" ("index")
2022-06-27 20:43:31,258 INFO sqlalchemy.engine.Engine [no key 0.00135s] ()
2022-06-27 20:43:31,269 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 20:43:31,276 INFO sqlalchemy.engine.E

In [191]:
df_links.to_sql("LinksEpisodes", sqlite_connection)

2022-06-27 20:43:31,329 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("LinksEpisodes")
2022-06-27 20:43:31,331 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:31,340 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("LinksEpisodes")
2022-06-27 20:43:31,344 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:31,348 INFO sqlalchemy.engine.Engine 
CREATE TABLE "LinksEpisodes" (
	"index" BIGINT, 
	shows_id BIGINT, 
	episode_id TEXT, 
	previousepisode_id TEXT, 
	nextepisode_id TEXT
)


2022-06-27 20:43:31,352 INFO sqlalchemy.engine.Engine [no key 0.00431s] ()
2022-06-27 20:43:31,364 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 20:43:31,366 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_LinksEpisodes_index" ON "LinksEpisodes" ("index")
2022-06-27 20:43:31,368 INFO sqlalchemy.engine.Engine [no key 0.00234s] ()
2022-06-27 20:43:31,380 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 20:43:31,384 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-27 20:43:31,

In [192]:
df_shows_genres.drop(columns=['combination']).to_sql("Generos", sqlite_connection)

2022-06-27 20:43:31,442 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Generos")
2022-06-27 20:43:31,444 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:31,447 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Generos")
2022-06-27 20:43:31,449 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 20:43:31,473 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Generos" (
	"index" BIGINT, 
	shows_id BIGINT, 
	"Action" SMALLINT, 
	"Adult" SMALLINT, 
	"Adventure" SMALLINT, 
	"Anime" SMALLINT, 
	"Children" SMALLINT, 
	"Comedy" SMALLINT, 
	"Crime" SMALLINT, 
	"DIY" SMALLINT, 
	"Drama" SMALLINT, 
	"Family" SMALLINT, 
	"Fantasy" SMALLINT, 
	"Food" SMALLINT, 
	"History" SMALLINT, 
	"Horror" SMALLINT, 
	"Legal" SMALLINT, 
	"Medical" SMALLINT, 
	"Music" SMALLINT, 
	"Mystery" SMALLINT, 
	"Nature" SMALLINT, 
	"Romance" SMALLINT, 
	"Science-Fiction" SMALLINT, 
	"Sports" SMALLINT, 
	"Supernatural" SMALLINT, 
	"Thriller" SMALLINT, 
	"Travel" SMALLINT, 
	"War" SMALLINT, 
	quantity BIGI